In [1]:
# 첨부파일 경로 변환 - DB 직접 연동 버전
# type 컬럼으로 게시판 선택

import json
import os
import shutil
from pathlib import Path
import pymysql
from sqlalchemy import create_engine

# ==================== DB 설정 ====================
USER = "root"
PASS = "1234"
HOST = "127.0.0.1"
PORT = 3306
DB = "crawling"

# ==================== 설정 영역 (여기를 수정하세요) ====================

# 게시판 타입 (필터링용)
BOARD_TYPE = "techInfo"  # ✨ 추가: hredu, cp, sales 등

# 첨부파일 원본 경로들 - 여러 경로 추가 가능
BASE_FILE_PATHS = [
    r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\file\brd",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\file\brd",
    r"C:\Users\LEEJUHWAN\Downloads\2011-01-01~2015-12-31\html\resource\file\brd",
    r"C:\Users\LEEJUHWAN\Downloads\2010-01-01~2010-12-31\html\resource\file\brd",
    # 더 추가하고 싶으면 여기에 계속 추가
]

# 첨부파일 복사본 저장 폴더 (타입별로 자동 생성)
OUTPUT_FILE_FOLDER = rf"C:\Users\LEEJUHWAN\Downloads\board_{BOARD_TYPE}_attachments"  # ✨ 타입 반영

# path에서 제거할 접두사 (이 부분을 제거하고 상대경로로 변경)
REMOVE_PREFIX = r"C:\Users\LEEJUHWAN\Downloads"

# ======================================================================


def find_attachment_folder(board_id):
    """여러 경로에서 첨부파일 폴더 찾기"""
    for base_path in BASE_FILE_PATHS:
        attachment_folder = Path(base_path) / board_id
        if attachment_folder.exists():
            return attachment_folder
    return None


def find_file_in_folder(folder, filename):
    """폴더 내에서 파일명과 일치하는 파일 찾기 (대소문자 구분 없음)"""
    if not folder or not folder.exists():
        return None
    
    # 정확한 파일명으로 먼저 찾기
    target_file = folder / filename
    if target_file.exists() and target_file.is_file():
        return target_file
    
    # 대소문자 구분 없이 찾기
    for file in folder.iterdir():
        if file.is_file() and file.name.lower() == filename.lower():
            return file
    
    return None


def process_single_post_attachments(data, post_index):
    """하나의 post의 첨부파일 처리"""
    
    # sourceId에서 게시판ID 추출
    source_id = data.get('sourceId', '')
    if not source_id:
        print(f"⚠️ [{post_index}] sourceId 없음, 건너뜀")
        return data
    
    board_id = source_id.lstrip('_')
    
    # attaches 배열 확인
    attaches = data.get('attaches', [])
    if not attaches:
        return data  # 첨부파일 없으면 그대로 반환
    
    # 여러 경로에서 첨부파일 폴더 찾기
    attachment_folder = find_attachment_folder(board_id)
    
    if not attachment_folder:
        print(f"⚠️ [{post_index}] 게시판ID {board_id} 폴더 없음 (모든 경로 검색 완료)")
        # 폴더가 없으면 모든 첨부파일의 path를 공란으로
        for attach in attaches:
            attach['path'] = ''
        return data
    
    # 복사본 저장할 게시판별 폴더 생성
    output_board_folder = Path(OUTPUT_FILE_FOLDER) / f"brd{board_id}"
    output_board_folder.mkdir(parents=True, exist_ok=True)
    
    print(f"\n📄 [{post_index}] 게시판ID: {board_id}")
    print(f"   원본: {attachment_folder}")
    print(f"   복사: {output_board_folder}")
    print(f"   ✅ 첨부파일 {len(attaches)}개 발견")
    
    # 각 첨부파일 처리
    processed_attaches = []
    copied_count = 0
    
    for idx, attach in enumerate(attaches):
        filename = attach.get('name', '')
        if not filename:
            print(f"      ⚠️ [{idx}] 파일명 없음 - path 공란 처리")
            attach['path'] = ''
            processed_attaches.append(attach)
            continue
        
        # 원본 폴더에서 파일 찾기
        source_file = find_file_in_folder(attachment_folder, filename)
        
        if source_file:
            # 복사본 파일 경로
            dest_file = output_board_folder / filename
            
            # 파일 복사
            try:
                shutil.copy2(source_file, dest_file)
                copied_count += 1
                print(f"      ✅ {filename}")
                
                # path를 상대경로로 변경 (REMOVE_PREFIX 제거)
                new_path = str(dest_file).replace('\\', '/')
                if REMOVE_PREFIX:
                    new_path = new_path.replace(REMOVE_PREFIX.replace('\\', '/'), '').lstrip('/')
                    new_path = new_path.replace(REMOVE_PREFIX, '').lstrip('\\')
                
                attach['path'] = new_path
                
            except Exception as e:
                print(f"      ❌ {filename} 복사 실패: {e}")
                attach['path'] = ''
        else:
            print(f"      ⚠️ 파일 없음: {filename} - path 공란 처리")
            attach['path'] = ''
        
        processed_attaches.append(attach)
    
    print(f"   📊 복사 완료: {copied_count}/{len(attaches)}개")
    
    # 수정된 attaches 적용
    data['attaches'] = processed_attaches
    return data


def process_db(board_type):
    """DB에서 특정 type의 데이터만 읽고 처리 후 DB에 업데이트"""
    
    # 출력 폴더 생성
    os.makedirs(OUTPUT_FILE_FOLDER, exist_ok=True)
    
    # DB 연결
    print(f"📂 DB 연결 중...")
    print(f"📂 게시판 타입: {board_type}")  # ✨ 추가
    print(f"📂 검색할 첨부파일 경로: {len(BASE_FILE_PATHS)}개\n")
    
    connection = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASS,
        database=DB,
        port=PORT,
        charset='utf8mb4'
    )
    
    try:
        with connection.cursor(pymysql.cursors.DictCursor) as cursor:
            # 1. DB에서 특정 type의 데이터만 읽기
            query = "SELECT * FROM board_posts WHERE type = %s"  # ✨ WHERE 조건 추가
            cursor.execute(query, (board_type,))
            posts = cursor.fetchall()
            
            total_count = len(posts)
            print(f"✅ 총 {total_count}개의 posts 발견\n")
            
            # 2. 각 post 처리
            success_count = 0
            total_attachments = 0
            
            for idx, post in enumerate(posts, 1):
                try:
                    # attaches_json을 파싱하여 attaches로 변환
                    if post.get('attaches_json'):
                        post['attaches'] = json.loads(post['attaches_json'])
                    else:
                        post['attaches'] = []
                    
                    # 첨부파일이 있는 경우만 카운트
                    if post.get('attaches'):
                        total_attachments += len(post.get('attaches'))
                    
                    # 기존 로직 그대로 사용
                    result = process_single_post_attachments(post, idx)
                    
                    # attaches가 변경된 경우 DB 업데이트
                    if result.get('attaches'):
                        # attaches를 JSON 문자열로 변환
                        attaches_json = json.dumps(result['attaches'], ensure_ascii=False)
                        
                        update_query = """
                            UPDATE board_posts 
                            SET attaches_json = %s 
                            WHERE sourceId = %s AND type = %s
                        """  # ✨ WHERE 조건에 type도 추가
                        cursor.execute(update_query, (attaches_json, result['sourceId'], board_type))
                        success_count += 1
                    
                except Exception as e:
                    print(f"❌ [Post {idx}] 처리 오류: {e}")
                    continue
            
            # 변경사항 커밋
            connection.commit()
            
            print(f"\n{'='*60}")
            print(f"✅ 처리 완료!")
            print(f"   - 게시판 타입: {board_type}")  # ✨ 추가
            print(f"   - 첨부파일 있는 posts: {success_count}개")
            print(f"   - 총 첨부파일 개수: {total_attachments}개")
            print(f"   - 복사된 파일 위치: {OUTPUT_FILE_FOLDER}")
            print(f"   - DB 테이블: board_posts (attaches_json 컬럼 업데이트)")
            print(f"{'='*60}")
    
    finally:
        connection.close()
        print("\n✅ DB 연결 종료")


# 실행
if __name__ == "__main__":
    process_db(BOARD_TYPE)  # ✨ board_type 인자 추가

📂 DB 연결 중...
📂 게시판 타입: techInfo
📂 검색할 첨부파일 경로: 4개

✅ 총 185개의 posts 발견


📄 [1] 게시판ID: 4075480
   원본: C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\file\brd\4075480
   복사: C:\Users\LEEJUHWAN\Downloads\board_techInfo_attachments\brd4075480
   ✅ 첨부파일 1개 발견
      ✅ (2025년_개정판)_SW사업_대가산정_가이드.pdf
   📊 복사 완료: 1/1개

📄 [2] 게시판ID: 3831102
   원본: C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\file\brd\3831102
   복사: C:\Users\LEEJUHWAN\Downloads\board_techInfo_attachments\brd3831102
   ✅ 첨부파일 3개 발견
      ✅ (2024년_개정판)_SW사업_대가산정_가이드(20240513).pdf
      ✅ 1101.SW사업 구현단계_SW개발비_간이법_산정양식_240513.xlsx
      ✅ 2401.SW사업 운영단계_SW재개발비_간이법_산정양식_240513.xlsx
   📊 복사 완료: 3/3개

📄 [3] 게시판ID: 3751452
   원본: C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\file\brd\3751452
   복사: C:\Users\LEEJUHWAN\Downloads\board_techInfo_attachments\brd3751452
   ✅ 첨부파일 1개 발견
      ✅ 2024 지식재산권의 손쉬운 이용.pdf
   📊 복사 완료: 1/1개

📄 [5] 게시판ID: 3396654
   원본: C:\Users\LEEJUHWAN\Downloads